In [5]:
import boto3
from botocore.exceptions import ClientError


def get_secret(secret_name):
    if secret_name not in ["DataSource_PostgresDB_Credentials","DataTarget_PostgresDB_Credentials"]:
        if type(secret_name) == str:
            raise ValueError
        else:
            raise TypeError

    region_name = "eu-west-2"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(service_name="secretsmanager", region_name=region_name)

    try:
        get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    except ClientError as e:
        raise e

    secret = get_secret_value_response["SecretString"]
    return secret


In [6]:
import boto3
from botocore.exceptions import ClientError
from pg8000.native import Connection, Error
import pandas as pd
import json
import os

if os.environ.get("AWS_EXECUTION_ENV") is not None:
    from get_secret import get_secret
else:
    from src.load.get_secret import get_secret


def create_connection(stage_name):
    # Retrieve the secret
    if stage_name == "extraction":
        secret = get_secret("DataSource_PostgresDB_Credentials")
    elif stage_name == "load":
        secret = get_secret("DataTarget_PostgresDB_Credentials")
    secret_dict = json.loads(secret)

    # Extract credentials from the secret
    user = secret_dict["user"]
    password = secret_dict["password"]
    host = secret_dict["host"]
    database = secret_dict["database"]
    port = secret_dict["port"]

    # Establish the database connection
    conn = Connection(
        user=user, database=database, host=host, password=password, port=port
    )
    return conn

In [7]:
def have_a_look_at_the_warehouse():
    db = create_connection("load")
    are_there_tables_query = """
    SELECT table_schema , table_name 
    FROM information_schema.tables
    WHERE table_schema not in ('information_schema', 'pg_catalog')
        AND table_type = 'BASE TABLE'
    ORDER BY table_schema, table_name
    """ # the answer was yes


    tables = db.run(are_there_tables_query)
    # print("Tables in the database:")
    for schema, table_name in tables:
        print(f"- {schema}.{table_name}")
    sql_query = """
    SELECT * from project_team_9.fact_sales_order LIMIT 10;
    """
    response = db.run(sql_query)
    columns = db.columns
    print(response, columns)

if __name__ == "__main__":
    have_a_look_at_the_warehouse()


Tables in the database:
- project_team_9.dim_counterparty
- project_team_9.dim_currency
- project_team_9.dim_date
- project_team_9.dim_design
- project_team_9.dim_location
- project_team_9.dim_payment_type
- project_team_9.dim_staff
- project_team_9.dim_transaction
- project_team_9.fact_payment
- project_team_9.fact_purchase_order
- project_team_9.fact_sales_order
[] [{'table_oid': 404637, 'column_attrnum': 1, 'type_oid': 23, 'type_size': 4, 'type_modifier': -1, 'format': 0, 'name': 'sales_record_id'}, {'table_oid': 404637, 'column_attrnum': 2, 'type_oid': 23, 'type_size': 4, 'type_modifier': -1, 'format': 0, 'name': 'sales_order_id'}, {'table_oid': 404637, 'column_attrnum': 3, 'type_oid': 1082, 'type_size': 4, 'type_modifier': -1, 'format': 0, 'name': 'created_date'}, {'table_oid': 404637, 'column_attrnum': 4, 'type_oid': 1083, 'type_size': 8, 'type_modifier': -1, 'format': 0, 'name': 'created_time'}, {'table_oid': 404637, 'column_attrnum': 5, 'type_oid': 1082, 'type_size': 4, 'type_m